In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dropout,Flatten,Dense

In [13]:
# train 데이터 로드
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이터/mnist/train.csv',index_col=0)
x_train = train_df.iloc[:,1:]
y_train = train_df.iloc[:,0]
x_train = x_train.astype('float32')/255.     # 데이터 정규화
y_train = tf.keras.utils.to_categorical(y_train,10)

from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(x_train,y_train,random_state = 32)
x_train.shape, y_train.shape

((45000, 784), (45000, 10))

In [14]:
from tensorflow.keras.applications.resnet50 import  ResNet50
resnet50 = ResNet50(include_top=False,weights='imagenet', input_shape=(32,32,3))

In [ ]:
resnet50.summary()

In [16]:
inputs = tf.keras.Input(shape=(28,28,1))
x = Conv2D(32,3,activation='relu')(inputs)
x = tf.keras.layers.BatchNormalization()(x)
x = MaxPooling2D(2)(x)
x = Conv2D(16,3,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = MaxPooling2D(2)(x)
x = Conv2D(32,3,activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = MaxPooling2D(2)(x)
x = Flatten()(x)
x = Dense(64,activation='relu')(x)
outputs = Dense(10,activation='softmax')(x)

In [18]:
model = tf.keras.Model(inputs,outputs)
callback = [
    tf.keras.callbacks.EarlyStopping(patience=5)
]
model.compile(loss='mse',
              optimizer='adam',metrics=['acc'])
x_train_reshape = x_train.to_numpy().reshape(-1,28,28,1)
x_test_reshape = x_test.to_numpy().reshape(-1,28,28,1)
hisy = model.fit(x_train_reshape,y_train, batch_size = 126, epochs = 200, callbacks = callback, validation_data = (x_test_reshape,y_test))

Epoch 1/200
358/358 [==============================] - 3s 8ms/step - loss: 0.0017 - acc: 0.9895 - val_loss: 0.0032 - val_acc: 0.9792
Epoch 2/200
358/358 [==============================] - 2s 6ms/step - loss: 0.0016 - acc: 0.9905 - val_loss: 0.0028 - val_acc: 0.9819
Epoch 3/200
358/358 [==============================] - 2s 6ms/step - loss: 0.0015 - acc: 0.9912 - val_loss: 0.0038 - val_acc: 0.9761
Epoch 4/200
358/358 [==============================] - 2s 6ms/step - loss: 0.0013 - acc: 0.9922 - val_loss: 0.0033 - val_acc: 0.9793
Epoch 5/200
358/358 [==============================] - 2s 6ms/step - loss: 0.0010 - acc: 0.9937 - val_loss: 0.0031 - val_acc: 0.9801
Epoch 6/200
358/358 [==============================] - 2s 6ms/step - loss: 0.0011 - acc: 0.9939 - val_loss: 0.0032 - val_acc: 0.9802
Epoch 7/200
358/358 [==============================] - 2s 6ms/step - loss: 9.4335e-04 - acc: 0.9946 - val_loss: 0.0035 - val_acc: 0.9780


In [35]:
# 모델이 완성되었으니 제공된 test를 돌려서 제출
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이터/mnist/test.csv',index_col=0)
x_test = x_test.astype('float32')/255.   # 데이터 정규화

In [36]:
test_reshape = test_df.to_numpy().reshape(-1,28,28,1)
test_reshape.shape

(10000, 28, 28, 1)

In [37]:
len(test_df)

10000

In [39]:
result = model.predict(test_reshape)

313/313 [==============================] - 1s 2ms/step


In [40]:
submission_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/데이터/mnist/sample_submission.csv',index_col = 0)

In [41]:
submission_df['label'] = [np.argmax(i) for i in result]

In [43]:
submission_df.to_csv('submission.csv')